## Amangeldi Nurgalym - DES

## 2. Реализация простого шифрования и расшифровки

In [1]:
# Импорт библиотек и констант из файла des_constants
import numpy as np
from des_constants import *

# Функция расширения половины блока до полного размера
def expand(half_block: str):
    return "".join(half_block[expansion_table[i]] for i in range(48))

# Функция для выполнения операции "исключающее ИЛИ" над двумя строками битов
def xor(str1: str, str2: str):
    return "".join(map(lambda c: '0' if c[0] == c[1] else '1', zip(str1, str2)))

# Счетчик для подсчета использования каждого S-блока
ct = [0]*64

# Реализация одного раунда шифрования Фейстеля
def feistel_fn(half_block: str, subkey: str):
    # Расширение половины блока
    expanded = expand(half_block)

    # Применение операции "исключающее ИЛИ" с подключом
    result = xor(expanded, subkey)

    # Применение S-блоков
    res = "".join(substitution_boxes[i//6][int(result[i] + result[i+5] + result[i+1:i+5], base=2)] for i in range(0, 48, 6))
    
    # Увеличение счетчика использования S-блоков
    ct[int(result[24:30], base=2)] += 1

    # Перестановка результата
    final_ans = "".join(res[feistel_permutation[i]] for i in range(32))

    return final_ans

# Генерация подключей для каждого раунда шифрования DES
def generate_subkeys(key: str, rounds: int):
    ans = [None]*rounds
    rotated = 0
    for i in range(rounds):
        if i in [0, 1, 8, 15]:
            rotated += 1
        else:
            rotated += 2
        ans[i] = [(rotated+k)%28 for k in range(27)] + [(rotated-1)%28] + [28+(rotated+k)%28 for k in range(27)] + [28+(rotated-1)%28]
        ans[i] = [ans[i][elem] for elem in pc2][:48]
    return ans

# Функция DES, выполняющая шифрование для одного блока данных
def des(block: str, key: str, rounds, subkey_table):
    # Перестановка блока
    block = "".join(block[elem] for elem in permutation1)
    
    # Генерация подключей для сети Фейстеля
    subkeys = []
    for subkey in subkey_table:
        ans = "".join(key[idx] for idx in subkey)
        subkeys.append(ans)

    # Применение раундов шифрования
    left = block[:32]; right = block[32:]
    for i in range(rounds):
        tmp = right
        right = xor(left, feistel_fn(right, subkeys[i]))
        left = tmp
    block = right + left
    
    # Применение обратной перестановки к блоку и возврат результата
    ciphertext = block
    ciphertext = "".join(block[elem] for elem in inv_permutation1)
    return ciphertext

# Функция для шифрования/дешифрования данных
def edcrypt(plain_or_cipher: str, key: str, rounds: int=16, decrypt=False):
    # Выравнивание данных до размера, кратного 64 битам
    floor = (len(plain_or_cipher) >> 6) << 6
    if len(plain_or_cipher) != floor:
        plain_or_cipher = plain_or_cipher + '1' + '0' * (floor + 63 - len(plain_or_cipher))
        floor += 64

    # Получение основного ключа
    key = "".join(key[elem] for elem in pc1)[:56]

    # Генерация подключей для каждого раунда
    subkey_table = generate_subkeys(key, rounds)

    # Шифрование/дешифрование блоков данных
    return "".join(des(block=plain_or_cipher[i:i+64], key=key, rounds=rounds, subkey_table=list(reversed(subkey_table)) if decrypt else subkey_table) for i in range(0, floor, 64))

# Функции для удобного интерфейса шифрования и дешифрования
def encrypt(plaintext: str, key: str, rounds: int=16):
    return edcrypt(plaintext, key, rounds=rounds, decrypt=False)

def decrypt(ciphertext: str, key: str, rounds: int=16):
    return edcrypt(ciphertext, key, rounds=rounds, decrypt=True)

# Основной код программы
if __name__ == "__main__":
    # Задание входных данных
    plaintext = "1010101111001101111001101010101111001101000100110010010100110110"
    key = "1010101010111011000010010001100000100111001101101100110011011101"

    # Шифрование и дешифрование
    cipher = encrypt(plaintext, key)
    decrypted = decrypt(cipher, key)

    # Вывод результатов
    print("Зашифрованный текст:", cipher)
    print("Расшифрованный текст:", decrypted)

    # Проверка соответствия ожидаемым результатам
    expected_output = "1001111000100110100111110101101011111010010011011011101101110000"
    print("Соответствие зашифрованного текста ожидаемому результату:", cipher == expected_output)
    print("Соответствие расшифрованного текста исходному:", plaintext == decrypted)


Зашифрованный текст: 1001111000100110100111110101101011111010010011011011101101110000
Расшифрованный текст: 1010101111001101111001101010101111001101000100110010010100110110
Соответствие зашифрованного текста ожидаемому результату: True
Соответствие расшифрованного текста исходному: True
